# [E-13] 프로젝트 - Movielens 영화 추천 실습

## 조건
* 유저가 영화에 대해 평점을 매긴 데이터가 데이터 크기 별로 있습니다. MovieLens 1M Dataset 사용을 권장합니다  
* 별점 데이터는 대표적인 explicit 데이터입니다. 하지만 implicit 데이터로 간주하고 테스트해 볼 수 있습니다.  
* 별점을 시청횟수로 해석해서 생각하겠습니다.  
* 또한 유저가 3점 미만으로 준 데이터는 선호하지 않는다고 가정하고 제외하겠습니다.

## Import

In [1]:
import numpy as np
import pandas as pd
from scipy.sparse import csr_matrix
from implicit.als import AlternatingLeastSquares

import os

## Data Load

In [2]:
file_path = os.getenv('HOME') + '/aiffel/recommendata_iu/data/ml-1m/ratings.dat'
ratings_cols = ['user_id', 'movie_id', 'ratings', 'timestep']
ratings = pd.read_csv(file_path, sep='::', names=ratings_cols, engine='python', encoding = "ISO-8859-1")
original_data_size = len(ratings)
ratings.head()

,user_id,movie_id,ratings,timestep
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


### 조건 4 (3점 미만 데이터는 선호하지 않음)

In [3]:
ratings = ratings[ratings['ratings']>=3]
filtered_data_size = len(ratings)

print("original_data_size : {0}, filtered_data_size : {1}".format(original_data_size, filtered_data_size))
print("Ratio of Remaining Data is {0:.2%}".format(filtered_data_size / original_data_size))

original_data_size : 1000209, filtered_data_size : 836478
Ratio of Remaining Data is 83.63%


### 조건3 (별점을 시청횟수로 해석)

In [4]:
ratings.rename(columns={'ratings':'counts'}, inplace=True) # 칼럼의 이름 변경: ratings -> counts

ratings['counts']

0          5
1          3
2          3
3          4
4          5
          ..
1000203    3
1000205    5
1000206    5
1000207    4
1000208    4
Name: counts, Length: 836478, dtype: int64

### Meta Data Load

In [9]:
movie_file_path=os.getenv('HOME') + '/aiffel/recommendata_iu/data/ml-1m/movies.dat'
columns = ['movie_id', 'title', 'genre'] 
meta_movies = pd.read_csv(movie_file_path, sep='::', names=columns, engine='python', encoding='ISO-8859-1')
meta_movies.head()

,movie_id,title,genre
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy


### 데이터 합치기

In [10]:
on_data = pd.merge(ratings, meta_movies)
on_data

,user_id,movie_id,counts,timestep,title,genre
0,1,1193,5,978300760,One Flew Over the Cuckoo's Nest (1975),Drama
1,2,1193,5,978298413,One Flew Over the Cuckoo's Nest (1975),Drama
2,12,1193,4,978220179,One Flew Over the Cuckoo's Nest (1975),Drama
3,15,1193,4,978199279,One Flew Over the Cuckoo's Nest (1975),Drama
4,17,1193,5,978158471,One Flew Over the Cuckoo's Nest (1975),Drama
...,...,...,...,...,...,...
836473,5851,3607,5,957756608,One Little Indian (1973),Comedy|Drama|Western
836474,5854,3026,4,958346883,Slaughterhouse (1987),Horror
836475,5854,690,3,957744257,"Promise, The (Versprechen, Das) (1994)",Romance
836476,5938,2909,4,957273353,"Five Wives, Three Secretaries and Me (1998)",Documentary


In [11]:
on_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 836478 entries, 0 to 836477
Data columns (total 6 columns):
 #   Column    Non-Null Count   Dtype 
---  ------    --------------   ----- 
 0   user_id   836478 non-null  int64 
 1   movie_id  836478 non-null  int64 
 2   counts    836478 non-null  int64 
 3   timestep  836478 non-null  int64 
 4   title     836478 non-null  object
 5   genre     836478 non-null  object
dtypes: int64(4), object(2)
memory usage: 44.7+ MB


### 소문자 전환

In [12]:
on_data['title'] = on_data['title'].str.lower()
on_data['genre'] = on_data['genre'].str.lower()
on_data.head(10)

,user_id,movie_id,counts,timestep,title,genre
0,1,1193,5,978300760,one flew over the cuckoo's nest (1975),drama
1,2,1193,5,978298413,one flew over the cuckoo's nest (1975),drama
2,12,1193,4,978220179,one flew over the cuckoo's nest (1975),drama
3,15,1193,4,978199279,one flew over the cuckoo's nest (1975),drama
4,17,1193,5,978158471,one flew over the cuckoo's nest (1975),drama
5,18,1193,4,978156168,one flew over the cuckoo's nest (1975),drama
6,19,1193,5,982730936,one flew over the cuckoo's nest (1975),drama
7,24,1193,5,978136709,one flew over the cuckoo's nest (1975),drama
8,28,1193,3,978125194,one flew over the cuckoo's nest (1975),drama
9,33,1193,5,978557765,one flew over the cuckoo's nest (1975),drama


## 2) 분석해 봅시다.  
* ratings에 있는 유니크한 영화 개수  
* ratings에 있는 유니크한 사용자 수  
* 가장 인기 있는 영화 30개(인기순)

In [38]:
ratings_u_unique = ratings['user_id'].unique()
ratings_m_unique = ratings['movie_id'].unique()

print(len(ratings_u_unique))
print(len(ratings_m_unique))

6039
3628


In [14]:
movies_count = on_data.groupby('title')['user_id'].count()
movies_count.sort_values(ascending=False).head(30)

title
american beauty (1999)                                   3211
star wars: episode iv - a new hope (1977)                2910
star wars: episode v - the empire strikes back (1980)    2885
star wars: episode vi - return of the jedi (1983)        2716
saving private ryan (1998)                               2561
terminator 2: judgment day (1991)                        2509
silence of the lambs, the (1991)                         2498
raiders of the lost ark (1981)                           2473
back to the future (1985)                                2460
matrix, the (1999)                                       2434
jurassic park (1993)                                     2413
sixth sense, the (1999)                                  2385
fargo (1996)                                             2371
braveheart (1995)                                        2314
men in black (1997)                                      2297
schindler's list (1993)                                  2257
pr

## 3) 내가 선호하는 영화를 5가지 골라서 ratings에 추가해 줍시다.

In [39]:
search = meta_movies['title'] == 'Terminator 2: Judgment Day (1991)'
search = meta_movies[search]
search

,movie_id,title,genre
585,589,Terminator 2: Judgment Day (1991),Action|Sci-Fi|Thriller


In [16]:
my_favorite = ['toy story (1995)' , 'sabrina (1995)' ,'tom and huck (1995)' ,'terminator 2: judgment day (1991)' ,'back to the future (1985)']
m_genre = ["Animation|Children's|Comedy", "Comedy|Romance", "Adventure|Children's", "Action|Sci-Fi|Thriller", "Comedy|Sci-Fi"]
m_movie_id = [1, 7, 8, 589, 1250]
my_playlist = pd.DataFrame({'user_id': [941016]*5, 'title': my_favorite, 'genre': m_genre, 'movie_id': m_movie_id, 'counts':[5]*5})

if not on_data.isin({'user_id':[941016]})['user_id'].any(): 
    on_data = on_data.append(my_playlist)   

on_data.tail(10)

,user_id,movie_id,counts,timestep,title,genre
836473,5851,3607,5,9.577566e+08,one little indian (1973),comedy|drama|western
836474,5854,3026,4,9.583469e+08,slaughterhouse (1987),horror
836475,5854,690,3,9.577443e+08,"promise, the (versprechen, das) (1994)",romance
836476,5938,2909,4,9.572734e+08,"five wives, three secretaries and me (1998)",documentary
836477,5948,1360,5,1.016564e+09,identification of a woman (identificazione di ...,drama
0,941016,1,5,NaN,toy story (1995),Animation|Children's|Comedy
1,941016,7,5,NaN,sabrina (1995),Comedy|Romance
2,941016,8,5,NaN,tom and huck (1995),Adventure|Children's
3,941016,589,5,NaN,terminator 2: judgment day (1991),Action|Sci-Fi|Thriller
4,941016,1250,5,NaN,back to the future (1985),Comedy|Sci-Fi


In [17]:
user_unique = on_data['user_id'].unique()
title_unique = on_data['title'].unique()

user_to_idx = {v:k for k,v in enumerate(user_unique)}
title_to_idx = {v:k for k,v in enumerate(title_unique)}

print(user_to_idx[941016])
print(title_to_idx['toy story (1995)'])

6039
40


In [18]:
temp_user_data = on_data['user_id'].map(user_to_idx.get).dropna()
if len(temp_user_data) == len(on_data):   # 모든 row가 정상적으로 인덱싱되었다면
    print('user_id column indexing OK!!')
    on_data['user_id'] = temp_user_data   # data['user_id']을 인덱싱된 Series로 교체해 줍니다. 
else:
    print('user_id column indexing Fail!!')

# artist_to_idx을 통해 artist 컬럼도 동일한 방식으로 인덱싱해 줍니다. 
temp_title_data = on_data['title'].map(title_to_idx.get).dropna()
if len(temp_title_data) == len(on_data):
    print('title column indexing OK!!')
    on_data['title'] = temp_title_data
else:
    print('title column indexing Fail!!')

on_data

user_id column indexing OK!!
title column indexing OK!!


,user_id,movie_id,counts,timestep,title,genre
0,0,1193,5,978300760.0,0,drama
1,1,1193,5,978298413.0,0,drama
2,2,1193,4,978220179.0,0,drama
3,3,1193,4,978199279.0,0,drama
4,4,1193,5,978158471.0,0,drama
...,...,...,...,...,...,...
0,6039,1,5,NaN,40,Animation|Children's|Comedy
1,6039,7,5,NaN,516,Comedy|Romance
2,6039,8,5,NaN,1980,Adventure|Children's
3,6039,589,5,NaN,92,Action|Sci-Fi|Thriller


## 4) CSR matrix를 직접 만들어 봅시다.

In [40]:
csr_data = csr_matrix((on_data.counts, (on_data.user_id, on_data.movie_id)))
csr_data

<6040x3953 sparse matrix of type '<class 'numpy.int64'>'
	with 836483 stored elements in Compressed Sparse Row format>

## 5) als_model = AlternatingLeastSquares 모델을 직접 구성하여 훈련시켜 봅시다.

In [20]:
# implicit 라이브러리에서 권장하고 있는 부분
os.environ['OPENBLAS_NUM_THREADS']='1'
os.environ['KMP_DUPLICATE_LIB_OK']='True'
os.environ['MKL_NUM_THREADS']='1'

In [21]:
# Implicit AlternatingLeastSquares 모델 선언
als_model = AlternatingLeastSquares(factors=100, regularization=0.01, use_gpu=False, iterations=15, dtype=np.float32)

In [22]:
csr_data_transpose = csr_data.T
csr_data_transpose

<3953x6040 sparse matrix of type '<class 'numpy.int64'>'
	with 836483 stored elements in Compressed Sparse Column format>

In [23]:
als_model.fit(csr_data_transpose)

  0%|          | 0/15 [00:00<?, ?it/s]

## 6) 내가 선호하는 5가지 영화 중 하나와 그 외의 영화 하나를 골라 훈련된 모델이 예측한 나의 선호도를 파악해 보세요.

In [24]:
jongin, toystory = user_to_idx[941016], title_to_idx['toy story (1995)']
jongin_vector, toystory_vector = als_model.user_factors[jongin], als_model.item_factors[toystory]

In [25]:
jongin_vector

array([ 8.26952383e-02, -6.39586926e-01,  6.11850023e-02, -6.22176588e-01,
       -3.10445596e-02,  1.94764867e-01, -8.92985538e-02,  7.50813544e-01,
        1.01844035e-01,  3.32128972e-01,  5.70070520e-02,  7.64781535e-02,
        3.85877222e-01, -2.75129706e-01,  4.17853221e-02,  1.38045490e-01,
        3.66071582e-01,  8.27991486e-01,  8.36456269e-02, -7.28167302e-04,
        2.44017035e-01,  8.21215630e-01, -5.54360263e-03, -7.89419711e-02,
        1.61210045e-01, -1.19738746e+00, -4.70198125e-01, -3.43630649e-02,
       -7.52937675e-01,  2.34842375e-02, -6.77652180e-01,  1.82007164e-01,
        3.03392839e-02, -6.80458993e-02,  1.85475558e-01,  4.51183051e-01,
       -2.20080495e-01, -4.44128811e-01,  3.92112523e-01, -2.06695065e-01,
        5.62118948e-01, -2.79512227e-01,  3.18750739e-01,  6.69550300e-02,
        2.42279887e-01, -4.22752127e-02, -9.76645648e-01,  3.89243782e-01,
        1.48799434e-01,  8.80064368e-02,  6.13108575e-01,  2.87580401e-01,
       -1.99603289e-01,  

In [26]:
toystory_vector

array([ 2.0221947e-03,  6.6396326e-04,  4.0941164e-03,  2.2344163e-03,
        5.2766451e-03,  8.0708414e-03,  8.0652572e-03,  1.0879729e-05,
        1.0063500e-02,  4.5771715e-03,  2.4552287e-03,  5.5077346e-03,
        1.2650546e-02, -8.5381896e-04, -6.7320623e-04,  4.0129079e-03,
        1.1680644e-02,  7.0188800e-03,  7.5657256e-03,  1.2243949e-04,
        1.6408139e-04,  2.0741830e-03, -1.3491101e-04,  1.5010755e-03,
        2.9007830e-03,  6.2067289e-04, -2.1538264e-03, -2.5678352e-03,
       -6.1850145e-04,  3.5095240e-03,  4.0303143e-03,  3.7288556e-03,
       -1.9875767e-03, -8.0317138e-03,  4.4823191e-03,  7.4947351e-03,
        6.7025893e-03,  3.4190384e-03,  6.2641758e-03,  3.4450006e-03,
        7.1360543e-03, -2.1572993e-03,  2.5073034e-03, -6.6100485e-03,
        1.0089380e-02,  8.3452753e-05,  9.2467861e-03,  4.8350212e-03,
        5.5364245e-03,  2.5824630e-03, -1.7285538e-03,  4.5125545e-03,
        8.6392602e-03,  8.2017825e-04,  4.4155624e-03,  1.0297126e-02,
      

In [27]:
terminator2 = title_to_idx['terminator 2: judgment day (1991)']
terminator2_vector = als_model.item_factors[terminator2]
np.dot(jongin_vector, terminator2_vector)

0.0075875977

## 7) 내가 좋아하는 영화와 비슷한 영화를 추천받아 봅시다.

### 비슷한 영화 찾아보기

In [28]:
favorite_list = 'toy story (1995)'
title_id = title_to_idx[favorite_list]
similar_list = als_model.similar_items(title_id, N=15)
similar_list

[(40, 0.99999994),
 (1728, 0.738816),
 (1650, 0.7373177),
 (767, 0.7364777),
 (2711, 0.7248861),
 (1725, 0.715718),
 (1399, 0.7084223),
 (3374, 0.7059733),
 (2341, 0.7051515),
 (3222, 0.70449275),
 (219, 0.69795),
 (2214, 0.6927961),
 (753, 0.69232374),
 (1510, 0.6916561),
 (584, 0.6914247)]

In [29]:
idx_to_title = {v:k for k,v in title_to_idx.items()}
[idx_to_title[i[0]] for i in similar_list]

['toy story (1995)',
 'black and white (1999)',
 'river wild, the (1994)',
 'medicine man (1992)',
 'smile like yours, a (1997)',
 'beach, the (2000)',
 'howling, the (1980)',
 'peanuts - die bank zahlt alles (1996)',
 'my bodyguard (1980)',
 'jerry springer: ringmaster (1998)',
 'pokémon the movie 2000 (2000)',
 'money train (1995)',
 'dog day afternoon (1975)',
 'parent trap, the (1998)',
 'star trek iii: the search for spock (1984)']

### 추천영화 함수 적용

In [30]:
def get_similar_list(movies_name: str):
    title_id = title_to_idx[movies_name]
    similar_list = als_model.similar_items(title_id)
    similar_list = [idx_to_title[i[0]] for i in similar_list]
    return similar_list

In [31]:
get_similar_list('terminator 2: judgment day (1991)')

['terminator 2: judgment day (1991)',
 'six degrees of separation (1993)',
 'patch adams (1998)',
 'blair witch project, the (1999)',
 'trust (1990)',
 'kramer vs. kramer (1979)',
 'mamma roma (1962)',
 'frequency (2000)',
 'funeral, the (1996)',
 'heartburn (1986)']

## 8) 내가 가장 좋아할 만한 영화들을 추천받아 봅시다.

In [32]:
user = user_to_idx[941016]
# recommend에서는 user*item CSR Matrix를 받습니다.
title_recommended = als_model.recommend(user, csr_data, N=20)
title_recommended

[(3114, 0.42805135),
 (1249, 0.32298952),
 (2571, 0.3212131),
 (480, 0.2734895),
 (110, 0.26518878),
 (2355, 0.2505411),
 (1580, 0.24305056),
 (1240, 0.23007728),
 (551, 0.22243446),
 (34, 0.22062032),
 (457, 0.2199234),
 (356, 0.20804015),
 (1265, 0.197525),
 (588, 0.18994233),
 (2028, 0.18964885),
 (1610, 0.1854386),
 (908, 0.184116),
 (3196, 0.18357435),
 (3175, 0.1809688),
 (2916, 0.16976361)]

In [33]:
[idx_to_title[i[0]] for i in title_recommended]

['bloodsport 2 (1995)',
 'how to be a player (1997)',
 "soldier's daughter never cries, a (1998)",
 'grand day out, a (1992)',
 'groundhog day (1993)',
 'blind date (1987)',
 'paper, the (1994)',
 'when a man loves a woman (1994)',
 'peter pan (1953)',
 'mulan (1998)',
 'nell (1994)',
 'legends of the fall (1994)',
 'crush, the (1993)',
 'roman holiday (1953)',
 'jean de florette (1986)',
 'friday the 13th part viii: jason takes manhattan (1989)',
 'metropolis (1926)',
 'confessional, the (le confessionnal) (1995)',
 'twice upon a yesterday (1998)',
 'omega code, the (1999)']

### 추천 이유 알아보기

In [36]:
bloodsport = title_to_idx['bloodsport 2 (1995)']
explain = als_model.explain(user, csr_data, itemid=bloodsport)
explain

(0.4230032128983794,
 [(1, 0.4173190966634835),
  (7, 0.02497016141051937),
  (1250, 0.02175104167841982),
  (8, -0.012899909716154675),
  (589, -0.02813717713788861)],
 (array([[ 0.59805249,  0.11100076,  0.11931466, ...,  0.13947269,
           0.11390734,  0.11092701],
         [ 0.06638428,  0.59574122,  0.10195698, ...,  0.08246829,
           0.10421602,  0.09922824],
         [ 0.07135643,  0.073984  ,  0.5935483 , ...,  0.09204605,
           0.07057235,  0.0882191 ],
         ...,
         [ 0.08341199,  0.06461134,  0.07968313, ...,  0.51437907,
          -0.00431868,  0.01597846],
         [ 0.06812257,  0.07472958,  0.06610447, ...,  0.06542731,
           0.53254756, -0.00091038],
         [ 0.06634018,  0.07142734,  0.07571453, ...,  0.0950118 ,
           0.06492474,  0.5221348 ]]),
  False))

In [37]:
[(idx_to_title[i[0]], i[1]) for i in explain[1]]

[('james and the giant peach (1996)', 0.4173190966634835),
 ('christmas story, a (1983)', 0.02497016141051937),
 ('live nude girls (1995)', 0.02175104167841982),
 ('snow white and the seven dwarfs (1937)', -0.012899909716154675),
 ('meet me in st. louis (1944)', -0.02813717713788861)]

favorite_list 에 적어뒀던 영화 5가지를 가지고 다른 영화를 추천했음을 확인했으며  
5가지의 영화의 유사도를 통해 
  
james and the giant peach (1996)  
christmas story, a (1983)  
live nude girls (1995)  
snow white and the seven dwarfs (1937)  
meet me in st. louis (1944)  
  
라는 영화 5가지가 bloodsport 2 (1995) 영화를 추천할 수 있게 기여를 한것 또한 확인했습니다.

## 참고문헌 Reference

EXPLORATION_GN5 내용  
디스코드 Ex. 13 | 질문있어요

## 회고

지금까지 프로젝트를 많이 해결하지 못해 많은 아쉬움이 있었습니다.    
이번 방학기간에 밀린 프로젝트를 전부 마무리 짓고 다음주 수업부터는 프로젝트를 밀리지 않는 방향으로  
시간을 사용해야겠다 생각했으며 프로젝트는 밀리면 감당 안된다는... 초반 퍼실님들의 말씀을 떠올리며.. 이번 프로젝트를 제출합니다.  
  
1) 데이터 준비와 전처리  
데이터 준비와 전처리 과정에선 조건에 따른 전처리 방법을 LMS에 나온 내용대로 진행 해봤으며  
추가적인 내용으론 영화의 제목들을 전부 소문자로 변경해줘 검색하기 편하게 바꿔주었고  
어떤식으로 전처리가 되는지의 과정을 이해하고 넘어갔던 부분입니다. 그리고 처음에는 Meta Data를 어떻게 사용하는걸까? 생각했었으나  
merge 를 활용, 기존 ratings data 와 결합하여 moive_id 를 title 로서 직관적으로 확인할 수 있게 진행했었고  
나름대로의 만족스러운 결과가 나온것을 확인할 수 있었습니다.  
  
2) 분석해 봅시다.  
unique() 라는 method를 활용해 고유값으로 확인했으며 고유값은 곧 중복되는 값을 제거한다는 뜻과 같다고 이해했던 부분이며  
겹치지 않는 각 영화의 갯수와 사용자의 수를 확인했고 가장 인기 있는 영화 30개도 merge 를 활용했던 부분 덕분에  
직관적으로 어떤 영화가 인기가 있었는지도 title 제목으로 직접 볼 수 있었습니다.  
  
3) 내가 선호하는 영화를 5가지 골라서 ratings에 추가해 줍시다.  
개인적으로... 선호하는 영화는 toy story 뿐이었으며 나머지 4가지 영화에 대해선 보지 않았었던 영화였습니다.  
(너무 옛날 영화에요!!)  
테스트를 위해 5가지 영화를 추가해줬으며 영화제목 뿐 아니라  
조건문을 활용하여 movie_id, genre 값에 대해서도 확인 후 같이 추가해줬으며 user_id 부분에선 생일을 입력시켜 추가했습니다.  
user_id 에 이름이 아닌 생일을 입력시킨 이유는 후에 CSR matrix를 만들 때 user_id 는 원래 int 값으로 이루어진 DataFrame 인데  
여기에 str 값으로 이름을 대입시키면 int 와 str 값의 충돌로 error가 나기 때문에 int 값의 생일로 입력을 해줬습니다.  
또한 모델에 입력할 vocab 과 비슷한 내용의 title to index 등의 내용을 unique 처리, index 값을 지정 및 key 와 value 의 값의 위치를  
변경하여 하나의 변수값으로 대입해줬으며 CSR matrix를 만들기 전 index 값으로 묶어둔 data 들이 unique 값으로 설정됐기 때문에  
기존 DataFrame 의 값도 unique 값으로 설정하여 CSR matrix 의 내용도 unique 값으로 나타날 수 있게 전처리 진행했습니다.  
  
4) CSR matrix를 직접 만들어 봅시다.  
처음엔 shape 값을 nunique(nuique 값의 갯수) 한 값으로 지정하여 실행 시켰을땐 error가 났었지만  
이후 디스코드를 통한 하주완 그루님의 도움으로 shape 값을 뺀 후 진행, 정상진행 되는것을 확인했으며  
조웅제 퍼실님의 shape 값 사용의 이유에 대해서도 알 수 있었던 내용이었습니다.  
  
5) als_model = AlternatingLeastSquares 모델을 직접 구성하여 훈련시켜 봅시다.  
LMS 내용을 토대로 만들어둔 CSR matrix를 모델에 넣어 구성 및 훈련을 진행 시켰습니다.  
  
6) 내가 선호하는 5가지 영화 중 하나와 그 외의 영화 하나를 골라 훈련된 모델이 예측한 나의 선호도를 파악해 보세요.  
toy story (1995)를 선호하여 toy story (1995)와 terminator2 의 선호도를 확인해봤습니다.  
  
7) 내가 좋아하는 영화와 비슷한 영화를 추천받아 봅시다.  
toy story (1995)를 가장 좋아하는 영화로 지정했으며 similar metho 를 직접 사용해 toy story와 연관성 있는  
영화 15가지를 직접 추천받아 봤으며 연관성 점수도 같이 확인 할 수 있었으며, 이전에 배웠던 hitmap 과 비슷한 내용으로  
생각하고 진행했던 내용이었습니다.  
  
8) 내가 가장 좋아할 만한 영화들을 추천받아 봅시다.  
bloodsport 2 (1995) 를.. 추천 받았는데 제목만 봐도.. 그렇게 좋아할 만한 영화는 아니었으나  
가장 높은 점수인 0.42805135 으로 나온것을 확인 할 수 있었으며  
영화추천의 경우는 사용자 기반으로 추천보단 아이템 기반으로 추천하는 것이 더 좋을 수 있다고  
생각하게 됐었던 내용이었습니다.